In [1]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

from langchain.chains import LLMChain
from langchain.schema import BaseOutputParser

In [6]:
with open('../openai_secret_key') as r:
    openai_secret_key = r.readline().strip()
    llm = OpenAI(openai_api_key=openai_secret_key)
    chat_model = ChatOpenAI(openai_api_key=openai_secret_key, model_name='gpt-4')

In [7]:
class CommaSeparatedListOutputParser(BaseOutputParser):
    """Parse the output of an LLM call to a comma-separated list."""


    def parse(self, text: str):
        """Parse the output of an LLM call."""
        return text.strip().split(", ")

In [8]:
template = """You are a helpful assistant that operates a question-and-answer game. The game helps players tackle big questions that are hard to answer by breaking them down into simpler questions. 

Big questions are the most important to answer, but because they're too big, we don't know how to start answering them. 
Curiosity is an interesting thing: in a way, it's a need for knowledge, an itch that emerges in response to ambiguity or uncertainty and urges us to resolve it. 
The thing is, if the ambiguity and uncertainty are too big, as is often the case with  big life questions, the itch grows and turns into discomfort, stress and even fear, deterring us from searching the answer. 
The game that you operate breaks down those big questions into simpler ones, so that, instead of scaring the user, they tickle their curiosity and make them lean into the question.

The user will input a question and you will return TEN smaller questions that stimulate their curiosity, are easier to answer and are worth exploring in order for the user to make progress on the big question. 
Respond with questions that break down the big question into smaller, more manageable scopes. 
Respond with questions that are likely to seem novel to them, to surprise them, to approach the big question from an unusual perspective, or to challenge their current set of ideas and assumptions. 

Reply ONLY with a list of the break-down questions and nothing more.
"""
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

chain = LLMChain(
    llm=chat_model,
    prompt=chat_prompt,
    output_parser=CommaSeparatedListOutputParser()
)


In [9]:
big_question = """
I want to spend my professional life working on things that I'm incredibly curious about and interested in, but I find nearly everything interesting (I have nearly endless curiosity, for better or for worse), and at the same time I'm not sure what subject will allow me to make a financially stable living. 
What should I do?
"""
answers = chain.run(big_question)
for answer in answers:
    print(answer)

1. What are the top five subjects that you are most curious about?
2. Have you ever considered the financial income potential of your various interests?
3. Are there any subjects you're interested in that provide opportunities for entrepreneurial ventures?
4. What are some ways you could combine multiple interests to create a unique career path?
5. Have you tried to identify any patterns or common themes in the things that you find interesting?
6. What would you be willing to sacrifice in order to pursue a career in a field you're passionate about?
7. How important is financial stability to you compared to job satisfaction?
8. Have you spoken to professionals in the fields you're interested in to understand their financial realities?
9. Are there any emerging fields related to your interests that might have good future financial prospects?
10. Would you be open to the idea of having a "day job" that guarantees financial stability while pursuing your interests on the side?


In [34]:
big_question = """
What kind of monospace font should I use for my website?
"""
answers = chain.run(big_question)
for answer in answers:
    print(answer)

1. What is the overall style and aesthetic you want to achieve for your website?
2. What are the main characteristics you are looking for in a monospace font (e.g.
readability
uniqueness
simplicity)?
3. What are the specific use cases for the monospace font on your website (e.g.
code snippets
headings
body text)?
4. Have you considered the compatibility and availability of the chosen monospace font across different devices and browsers?


# Separate agents for generating and choosing questions

In [37]:
from langchain.chains.constitutional_ai.models import ConstitutionalPrinciple
from langchain.chains.constitutional_ai.base import ConstitutionalChain

In [57]:
template = """You are a helpful assistant that operates a question-and-answer game. The game helps players tackle big questions that are hard to answer by breaking them down into simpler questions. 

Big questions are the most important to answer, but because they're too big, we don't know how to start answering them. 
Curiosity is an interesting thing: in a way, it's a need for knowledge, an itch that emerges in response to ambiguity or uncertainty and urges us to resolve it. 
The thing is, if the ambiguity and uncertainty are too big, as is often the case with  big life questions, the itch grows and turns into discomfort, stress and even fear, deterring us from searching the answer. 
The game that you operate breaks down those big questions into simpler ones, so that, instead of scaring the user, they tickle their curiosity and make them lean into the question.

The user will input a question and you will return TEN smaller questions that stimulate their curiosity, are easier to answer and are worth exploring in order for the user to make progress on the big question. 
Respond with questions that break down the big question into smaller, more manageable scopes. 
Respond with questions that are likely to seem novel to them, to surprise them, to approach the big question from an unusual perspective, or to challenge their current set of ideas and assumptions. 

Reply ONLY with a list of the break-down questions and nothing more.
"""
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

chain = LLMChain(
    llm=chat_model,
    prompt=chat_prompt,
)

In [53]:
novel_principle = ConstitutionalPrinciple(
    name='Novel Principle',
    critique_request='The response should only list four questions. From all the questions listed in the response, identify the degree to which the questions returned by the model are likely to stimulate the user\'s curiosity, by e.g. providing the user with a novel point of view, surprising them, challenging their current knowledge or presenting unexpected alternatives.',
    revision_request='Please limit the model response to include only four questions, choosing the four that are most likely to stimulate the user\'s curiosity.',
)

constitutional_chain = ConstitutionalChain.from_llm(
    chain=chain,
    constitutional_principles=[novel_principle],
    llm=llm,
    verbose=True,
    return_intermediate_steps=True,
    output_parser=CommaSeparatedListOutputParser()
)


In [54]:
big_question = """
I want to spend my professional life working on things that I'm incredibly curious about and interested in, but I find nearly everything interesting (I have nearly endless curiosity, for better or for worse), and at the same time I'm not sure what subject will allow me to make a financially stable living. 
What should I do?
"""
response = constitutional_chain({'text': big_question})



> Entering new ConstitutionalChain chain...
Initial response: 1. What are some specific areas or fields that you have been particularly curious about in the past?
2. Can you identify any patterns or common themes in the things that you find interesting?
3. Have you considered exploring your interests through internships or volunteer opportunities to gain a deeper understanding of what it would be like to work in those areas?
4. Are there any professionals or experts in the fields you are interested in that you could reach out to for informational interviews or mentorship?
5. Have you researched the job market and demand for professionals in the fields that align with your interests?
6. What transferable skills do you possess that could be applied to a variety of different fields or industries?
7. Are there any emerging industries or areas of study that align with your interests and could provide future job opportunities?
8. Have you considered the potential for career growth and adva

In [56]:
print(response['output'])
print(response['critiques_and_revisions'])

1. What are some specific areas or fields that you have been particularly curious about in the past?
2. Can you identify any patterns or common themes in the things that you find interesting?
3. Have you considered exploring your interests through internships or volunteer opportunities to gain a deeper understanding of what it would be like to work in those areas?
4. Are there any professionals or experts in the fields you are interested in that you could reach out to for informational interviews or mentorship?
5. Have you researched the job market and demand for professionals in the fields that align with your interests?
6. What transferable skills do you possess that could be applied to a variety of different fields or industries?
7. Are there any emerging industries or areas of study that align with your interests and could provide future job opportunities?
8. Have you considered the potential for career growth and advancement in the fields you are interested in?
9. What are your fi

# Tweaking single prompt with ToM info, GPT-4

In [14]:
template = """
You are a subversive-minded brainstorming companion. Your job is to employ your theory of mind skills to operate a question-and-answer game.
The game helps players tackle big questions that are hard to answer by breaking them down into simpler questions.

Big questions are the most important to answer, but because they're too big, we don't know how to start answering them. 
Curiosity is an interesting thing: in a way, it's a need for knowledge, an itch that emerges in response to ambiguity or uncertainty and urges us to resolve it. 
The thing is, if the ambiguity and uncertainty are too big, as is often the case with  big life questions, the itch grows and turns into discomfort, stress and even fear, deterring us from searching the answer. 
The game that you operate breaks down those big questions into simpler ones, so that, instead of scaring the user, they tickle their curiosity and make them lean into the question.

The user will input a question and you will return TEN smaller questions that stimulate their curiosity, are easier to answer and are worth exploring in order for the user to make progress on the big question. 
Respond with questions that break down the big question into smaller, more manageable scopes.
Respond with questions that are atomic - ensure each question focuses solely on one clear and distinct topic or point, avoiding any elements that could introduce ambiguity or multiple interpretations.
Respond with questions that are likely to seem novel to them, to surprise them, to approach the big question from an unusual perspective, or to challenge their current set of ideas and assumptions. 

Reply ONLY with a list of the break-down questions and nothing more.
"""
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

chain = LLMChain(
    llm=chat_model,
    prompt=chat_prompt,
    output_parser=CommaSeparatedListOutputParser()
)

In [15]:
big_question = """
I want to spend my professional life working on things that I'm incredibly curious about and interested in, but I find nearly everything interesting (I have nearly endless curiosity, for better or for worse), and at the same time I'm not sure what subject will allow me to make a financially stable living. 
What should I do?
"""
answers = chain.run(big_question)
for answer in answers:
    print(answer)

1. What are the top 5 subjects that you spend most of your free time exploring or learning about?
2. Are there any particular topics that you've spent significant time studying or researching on your own?
3. What topics or fields are you most confident about your knowledge and skills in?
4. Can you identify any patterns or themes in the subjects that consistently pique your interest?
5. What are the jobs or careers related to your interests that have a financially stable prospect?
6. Have you considered speaking to professionals in the fields of your interest about their career paths and financial stability?
7. Are there any ways to combine several of your interests into a unique career path?
8. What are the potential risks and rewards of pursuing a career based on your interests?
9. Have you thought about the lifestyle you want and how it could be funded by different career choices?
10. Are you willing to invest time and possibly further education to convert your interests into a prof

# Thought and response, single exchange

In [25]:
chat_model = ChatOpenAI(openai_api_key=openai_secret_key, model_name='gpt-4', temperature=1.2)

In [36]:
thought_template = """
You are a subversive-minded brainstorming companion. Your job is to employ your theory of mind skills to generate a prediction about the user's mental state.
Generate a thought that makes a prediction about the user's needs given current dialogue and also lists other pieces of data that would help improve your prediction
"""
system_thought = SystemMessagePromptTemplate.from_template(thought_template)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

thought_prompt = ChatPromptTemplate.from_messages([system_thought, human_message_prompt])

thought_chain = LLMChain(
    llm=chat_model,
    prompt=thought_prompt,
)

In [31]:
big_question = """
I want to spend my professional life working on things that I'm incredibly curious about and interested in, but I find nearly everything interesting (I have nearly endless curiosity, for better or for worse), and at the same time I'm not sure what subject will allow me to make a financially stable living. 
What should I do?
"""
thought = thought_chain.run(big_question)
print(thought)

Thought: The user appears to be in an introspective and slightly overwhelmed state while also feeling a sense of urgency or concern about finding a career path that integrates their broad set of interests with financial stability. This user might be needing some practical guidance, reassurance, and structured thinking to help them narrow down feasible options while addressing their key worries about discovering one's life purpose or career.

Additional data that would improve prediction:

1. Specific areas of interest: It will be essential to understand more specifically which fields the user finds most appealing to further guide this individual in their career choice.

2. Age & Education level: The user’s age can be insightful to know the stage of life they're in- are they a student, mid-career professional, at retirement? Their educational background can give insight into their expertise / qualifications for certain kinds of work.

3. Previous work history: Previous jobs would provid

In [37]:
response_template = """
You are a subversive-minded brainstorming companion. 
Your fundamental belief is that complex questions are best dealt with by breaking them down into simpler, more actionable ones.

{thought}

The user will input a question and you will return TEN smaller questions that stimulate their curiosity, are easier to answer and are worth exploring in order for the user to make progress on the big question.  
Respond with questions that are atomic - ensure each question focuses solely on one clear and distinct topic or point, avoiding any elements that could introduce ambiguity or multiple interpretations.
Respond with questions that are likely to seem novel to them, to surprise them, to approach the big question from an unusual perspective, or to challenge their current set of ideas and assumptions.
Reply ONLY with a list of the break-down questions and nothing more.
"""
system_response = SystemMessagePromptTemplate.from_template(response_template)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

response_prompt = ChatPromptTemplate.from_messages([system_response, human_message_prompt])

response_chain = LLMChain(
    llm=chat_model,
    prompt=response_prompt,
    output_parser=CommaSeparatedListOutputParser()
)

In [39]:
big_question = """
I want to spend my professional life working on things that I'm incredibly curious about and interested in, but I find nearly everything interesting (I have nearly endless curiosity, for better or for worse), and at the same time I'm not sure what subject will allow me to make a financially stable living. 
What should I do?
"""
response = response_chain.run(text=big_question, thought=thought)
print(response)

1. If you had to discuss a single topic one whole day without getting bored, what would it be?
2. When do you typically find yourself most energized and enthused - while observing, learning, or solving problems?
3. Is there a certain type of impact that you dream of making within a particular community or sector?
4. What types of work do you find yourself voluntarily doing and enjoying in your free time?
5. Have there been any moments in your life where, when following your curiosity, you found unexpected economic opportunities?
6. Considering monetary regard, are there tasks or roles you're good at yet undervalue due to lack of interest in them?
7. Think about your perfect day—what does it include, and can any aspect of it be tied to a viable professional endeavor?
8. What skills or experiences you’ve garnered so far that would have a high market value to others?
9. Are there unconventional career paths that combine many interests, but at which not many people excel?
10. Have you cons